In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import matplotlib.pyplot as plt
import random as rd
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
import random
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.utils import shuffle
import time
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import seaborn as sns
import traceback

# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Generate permutations
gen = torch.Generator()
gen.manual_seed(seed)


In [ ]:
path = "/content/drive/MyDrive/Thesis/Datasets/multiclass"

#Бинарная классификация (нейтральное состояние/состояние стресса)

In [ ]:
"""
Модель классификации с одним LSTM-слоем
"""
class LSTMStressBinDetector(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2):
        super().__init__()
        self.lstm1 = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.fc(out[:, -1, :])
        out = self.sig(out)
        return out

In [ ]:
"""
Функция загрузки сгенерированных датасетов
Используется стратификация и перемешивание. Выборка делится на 85/15 для тренировочной и тестовой выборки соответственно
"""
def data_processing_binclass(extract_path, window, step):
  if step == 1:
    path = f"{extract_path}/chest_dataset.pkl"
  else:
    path = f"{extract_path}/chest_dataset_{window}_{step}.pkl"

  try:
    with open(path, 'rb') as file:
      dataset = pickle.load(file, encoding='latin1')
  except FileNotFoundError:
    return None

  X_train, Y_train, X_test, Y_test = None, None, None, None
  for key in dataset:
    binclass_mask = np.where(dataset[key]['label'] != 2)
    y = dataset[key]['label'][binclass_mask]
    arrays = train_test_split(dataset[key]['data'][binclass_mask], y, random_state=42, test_size=0.15, stratify=y)
    X_train = np.concatenate((arrays[0], X_train), axis=0) if X_train is not None else arrays[0]
    X_test = np.concatenate((arrays[1], X_test), axis=0) if X_test is not None else arrays[1]
    Y_train = np.concatenate((arrays[2], Y_train), axis=0) if Y_train is not None else arrays[2]
    Y_test = np.concatenate((arrays[3], Y_test), axis=0) if Y_test is not None else arrays[3]

  X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
  X_test, Y_test = shuffle(X_test, Y_test, random_state=42)

  X_train = torch.tensor(X_train, dtype=torch.float32)
  X_test = torch.tensor(X_test, dtype=torch.float32)
  Y_train = torch.tensor(Y_train, dtype=torch.float32)
  Y_test = torch.tensor(Y_test, dtype=torch.float32)

  # Для обучающего набора
  train_mask = ~torch.isnan(X_train).any(dim=(1, 2))
  X_train_clean = X_train[train_mask]
  Y_train_clean = Y_train[train_mask]

  # Для тестового набора
  test_mask = ~torch.isnan(X_test).any(dim=(1, 2))
  X_test_clean = X_test[test_mask]
  Y_test_clean = Y_test[test_mask]

  return X_train_clean, Y_train_clean, X_test_clean, Y_test_clean

In [ ]:
"""
Функция обучения и валидации на одном фолде
"""
def binclass_train_model(X_train, Y_train, X_val, Y_val, input_size, num_epochs=300, patience=30):
    model = LSTMStressBinDetector(input_size)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, min_lr=1e-6)

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_state = None
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        outputs = model(X_train)
        loss = criterion(outputs.view(-1), Y_train)
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs.view(-1), Y_val)

        scheduler.step(val_loss)

        # Logging
        train_losses.append(loss.item())
        val_losses.append(val_loss.item())

        # Save best model for this fold
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

        # Early stopping for this fold
        if early_stop_counter >= patience and epoch > 50:
            print(f"Early stopping at epoch {epoch+1} (no improvement for {patience} epochs)")
            break

    # Load best weights for this fold
    model.load_state_dict(best_model_state)
    return model, train_losses, val_losses

"""
Функция кросс-валидирования с помощью 10-fold
"""
def binclass_training_with_cv(X_train, Y_train, X_test, Y_test, n_splits=10, num_epochs=300, patience=30):
    input_size = X_train.shape[2]
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_metrics = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, Y_train)):
        print(f"\n--- Fold {fold + 1}/{n_splits} ---")

        # Split data
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        Y_fold_train, Y_fold_val = Y_train[train_idx], Y_train[val_idx]

        # Train with early stopping per fold
        model, train_loss, val_loss = binclass_train_model(
            X_fold_train, Y_fold_train,
            X_fold_val, Y_fold_val,
            input_size,
            num_epochs=num_epochs,
            patience=patience
        )

        # Evaluate on test set (optional per fold)
        fold_test_metrics = binclass_evaluate(model, X_test, Y_test)
        fold_metrics.append(fold_test_metrics)
        print(f"Fold {fold + 1} Test Metrics:", fold_test_metrics)

    return fold_metrics

"""
Функция оценка качества обучения
"""
def binclass_evaluate(model, X, Y, plot_cm=True, class_names=None):
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        preds = (outputs > 0.5).float().cpu().numpy()
        probs = outputs.cpu().numpy()
        labels = Y.cpu().numpy()

    # Вычисляем Confusion Matrix
    cm = confusion_matrix(labels, preds)

    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds),
        'f1': f1_score(labels, preds),
        'auroc': roc_auc_score(labels, probs),
        "confusion_matrix": cm
    }

In [ ]:
"""
Функция запуска эксперимента для одного датасета
"""
def run_experiment(path, window, timestep):
    ans = data_processing_binclass(path, window, timestep)

    if ans is None:
      return None

    X_train, Y_train, X_test, Y_test = ans
    # Записываем в лог параметры
    with open(log_file, "a") as f:
      f.write(f"\n=== Window: {window}, Timestep: {timestep} ===\n")

    print(f"\nProcessing window={window}, timestep={timestep}")

    try:
      metrics = binclass_training_with_cv(X_train, Y_train, X_test, Y_test, n_splits=10)

      # Записываем в лог метрики
      with open(log_file, "a") as f:
        f.write(f"Metrics: {metrics}\n")
        f.write(f"Accuracy: {np.mean([fold['accuracy'] for fold in metrics]):.4f}, Precision: {np.mean([fold['precision'] for fold in metrics]):.4f}, Recall: {np.mean([fold['recall'] for fold in metrics]):.4f},  F1: {np.mean([fold['f1'] for fold in metrics]):.4f}, AUROC: {np.mean([fold['auroc'] for fold in metrics]):.4f}\n")

        print(f"Final Test Metrics: {np.mean([fold['accuracy'] for fold in metrics]):.4f}, Precision: {np.mean([fold['precision'] for fold in metrics]):.4f}, Recall: {np.mean([fold['recall'] for fold in metrics]):.4f},  F1: {np.mean([fold['f1'] for fold in metrics]):.4f}, AUROC: {np.mean([fold['auroc'] for fold in metrics]):.4f}\n")
      return metrics

    except Exception as e:
      with open(log_file, "a") as f:
        f.write(f"Error: {str(e)}\n")
        print(f"Error for window={window}, timestep={timestep}: {str(e)}")

In [ ]:
# Создаем папку для результатов (если нет)
!mkdir -p /content/drive/MyDrive/Thesis/Results

# Инициализируем переменные для сохранения
all_steps = None
all_results = None
log_file = "/content/drive/MyDrive/Thesis/Results/chest_binclass_log_v1.txt"

# Открываем файл для записи логов
with open(log_file, "a") as f:
    f.write("Training Log - " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n")

chest_binclass_dict = {}

chest_binclass_dict[(15, 1)] = run_experiment(path, 0, 1)
chest_binclass_dict[(30, 1)] = chest_binclass_dict[(15, 1)]
chest_binclass_dict[(45, 1)] = chest_binclass_dict[(15, 1)]

for window in range(15, 46, 15):
    for timestep in range(2, 9):
        chest_binclass_dict[(window, timestep)] = run_experiment(path, window, timestep)

with open('/content/drive/MyDrive/Thesis/Results/chest_binclass_v1.pkl', 'wb') as f:
    pickle.dump(chest_binclass_dict, f)

# Выводим информацию о сохраненных файлах
print("\nSaved files:")
print(f"- Training logs: {log_file}")
print(f"- All results: /content/drive/MyDrive/Thesis/Results/chest_binclass_v1.pkl")

#Многоклассовая классификация

In [ ]:
"""
Модель классификации с одним LSTM-слоем
"""
class LSTMStressMultiDetector1h(nn.Module):
    def __init__(self, input_size=15, hidden_size=200, num_layers=1, num_classes=3):
        super().__init__()
        # Первый LSTM слой
        self.first_lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.first_lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
"""
Модель классификации с 4 LSTM-слоями, 3 из которых скрытые
"""
class LSTMStressMultiDetector4h(nn.Module):
    def __init__(self, input_size=15, hidden_size=200, num_layers=4, num_classes=3):
        super().__init__()
        cur_hidden_size = hidden_size
        # Первый LSTM слой
        self.first_lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=cur_hidden_size,
            num_layers=1,
            batch_first=True
        )
        cur_hidden_size -= 50

        self.hid_lstm = []
        self.dropout = []
        for _ in range(num_layers - 2):
          self.hid_lstm.append(
              nn.LSTM(
            input_size=cur_hidden_size + 50,
            hidden_size=cur_hidden_size,
            num_layers=1,
            batch_first=True
            )
          )

          self.dropout.append(
              nn.Dropout(0.2)
          )

          cur_hidden_size -= 50

        self.last_lstm = nn.LSTM(
            input_size=cur_hidden_size + 50,
            hidden_size=cur_hidden_size,
            num_layers=1,
            batch_first=True
        )

        self.fc = nn.Linear(cur_hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.first_lstm(x)

        for i in range(len(self.hid_lstm)):
          out, _ = self.hid_lstm[i](out)
          out = self.dropout[i](out)

        out, _ = self.last_lstm(out)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
"""
Функция загрузки сгенерированных датасетов
Используется стратификация и перемешивание. Выборка делится на 85/15 для тренировочной и тестовой выборки соответственно
"""
def data_processing_multiclass(extract_path, window, step, class_number=3):
    if step == 1:
      path = f"{extract_path}/chest_acc_dataset.pkl"
    else:
      path = f"{extract_path}/chest_acc_dataset_{window}_{step}.pkl"
    try:
        with open(path, 'rb') as file:
            dataset = pickle.load(file, encoding='latin1')
    except Exception as e:
        print(e)
        return None

    X_train, Y_train, X_test, Y_test = None, None, None, None
    for key in dataset:
        y = np.eye(class_number)[dataset[key]['label'].astype(int)]
        arrays = train_test_split(dataset[key]['data'], y, random_state=42, test_size=0.15, stratify=y.argmax(axis=1))
        X_train = np.concatenate((X_train, arrays[0]), axis=0) if X_train is not None else arrays[0]
        X_test = np.concatenate((X_test, arrays[1]), axis=0) if X_test is not None else arrays[1]
        Y_train = np.concatenate((Y_train, arrays[2]), axis=0) if Y_train is not None else arrays[2]
        Y_test = np.concatenate((Y_test, arrays[3]), axis=0) if Y_test is not None else arrays[3]

    X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
    X_test, Y_test = shuffle(X_test, Y_test, random_state=42)

    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    Y_train = torch.tensor(Y_train, dtype=torch.float32)
    Y_test = torch.tensor(Y_test, dtype=torch.float32)

    # Для обучающего набора
    train_mask = ~torch.isnan(X_train).any(dim=(1, 2))
    X_train_clean = X_train[train_mask]
    Y_train_clean = Y_train[train_mask]

    # Для тестового набора
    test_mask = ~torch.isnan(X_test).any(dim=(1, 2))
    X_test_clean = X_test[test_mask]
    Y_test_clean = Y_test[test_mask]

    return X_train_clean, Y_train_clean, X_test_clean, Y_test_clean

In [ ]:
"""
Функция обучения и валидации на одном фолде
"""
def multiclass_train_model(X_train, Y_train, X_val, Y_val, input_size, num_epochs=300, patience=30, model_type="LSTM1"):
    if model_type == "LSTM1":
      model = LSTMStressMultiDetector1h(input_size, num_classes=3)
    elif model_type == "LSTM4":
      model = LSTMStressMultiDetector4h(input_size, num_classes=3)
    else:
      raise ValueError(f"There no model with type {model_type}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Изменяем функцию потерь на CrossEntropyLoss (не нужно softmax в модели)
    class_counts = torch.bincount(Y_train.argmax(dim=1))
    class_weights = 1. / class_counts.float()
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, min_lr=1e-6)

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_state = None
    early_stop_counter = 0

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        outputs = model(X_train)
        loss = criterion(outputs, Y_train.argmax(dim=1))  # Используем argmax для классов
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, Y_val.argmax(dim=1))

        scheduler.step(val_loss)

        # Logging
        train_losses.append(loss.item())
        val_losses.append(val_loss.item())

        # Save best model for this fold
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

        # Early stopping for this fold
        if early_stop_counter >= patience and epoch > 50:
            print(f"Early stopping at epoch {epoch+1} (no improvement for {patience} epochs)")
            break

    # Load best weights for this fold
    model.load_state_dict(best_model_state)
    return model, train_losses, val_losses, epoch

"""
Функция оценка качества обучения
"""
def multiclass_evaluate(model, X, Y, plot_cm=True, class_names=None):
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        preds = outputs.argmax(dim=1).cpu().numpy()
        logits = outputs.cpu().numpy()
        labels = Y.argmax(dim=1).cpu().numpy()

    probs = torch.softmax(outputs, dim=1).cpu().numpy()
    y_true_bin = label_binarize(labels, classes=np.unique(labels))

    # Вычисляем Confusion Matrix
    cm = confusion_matrix(labels, preds)

    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='weighted'),
        'recall': recall_score(labels, preds, average='weighted'),
        'f1': f1_score(labels, preds, average='weighted'),
        'auroc': roc_auc_score(y_true_bin, probs, multi_class='ovr', average='weighted'),
        'confusion_matrix': cm  # Возвращаем матрицу для дальнейшего анализа
    }

"""
Функция кросс-валидирования с помощью 10-fold
"""
def multiclass_training_with_cv(X_train, Y_train, X_test, Y_test, n_splits=5, num_epochs=300, patience=30, model_type="LSTM1"):
    input_size = X_train.shape[2]
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_metrics = []
    fold_epochs = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, Y_train.argmax(dim=1))):  # Используем argmax для стратификации
        print(f"\n--- Fold {fold + 1}/{n_splits} ---")

        # Split data
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        Y_fold_train, Y_fold_val = Y_train[train_idx], Y_train[val_idx]

        # Train with early stopping per fold
        model, train_loss, val_loss, epoch = multiclass_train_model(
            X_fold_train, Y_fold_train,
            X_fold_val, Y_fold_val,
            input_size,
            num_epochs=num_epochs,
            patience=patience,
            model_type=model_type
        )

        # Evaluate on test set (optional per fold)
        class_names = ['Baseline', 'Stress', 'Amusement']
        fold_test_metrics = multiclass_evaluate(model, X_test, Y_test, plot_cm=True, class_names=class_names)
        fold_metrics.append(fold_test_metrics)
        fold_epochs.append(epoch)
        print(f"Fold {fold + 1} Test Metrics:", fold_test_metrics)

    return fold_metrics, fold_epochs

In [ ]:
"""
Функция запуска эксперимента для одного датасета
"""
def run_multiclass_experiment(path, window, timestep, model_type="LSTM1"):
    ans = data_processing_multiclass(path, window, timestep)

    if not ans:
      return None

    X_train, Y_train, X_test, Y_test = ans

    with open(log_file, "a") as f:
        f.write(f"\n=== Window: {window}, Timestep: {timestep} ===\n")

    print(f"\nProcessing window={window}, timestep={timestep}")
    start_time = time.time()

    try:
      metrics, epochs = multiclass_training_with_cv(X_train, Y_train, X_test, Y_test, n_splits=10, model_type=model_type)

      elapsed = time.time() - start_time

      with open(log_file, "a") as f:
        f.write(f"Time: {elapsed:.2f} sec\n")
        f.write(f"Epochs: {', '.join(map(str, epochs))}")
        f.write(f"Metrics: {metrics}\n")
        print(f"Class distribution (train): {np.unique(Y_train.argmax(dim=1), return_counts=True)}")
        print(f"Class distribution (test): {np.unique(Y_test.argmax(dim=1), return_counts=True)}")

        print("Final Test Metrics:", metrics)
        print("Final epoch's number")

    except Exception as e:
      with open(log_file, "a") as f:
          f.write(f"Error: {str(e)}\n")
          print(f"Error for window={window}, timestep={timestep}: {str(e)}\n")
          traceback.print_exc()

1 LSTM-слой

In [ ]:
!mkdir -p /content/drive/MyDrive/Thesis/Results
all_steps = None
all_results = None
log_file = "/content/drive/MyDrive/Thesis/Results/multiclass_log_v1_4h.txt"

with open(log_file, "a") as f:
    f.write("Training Log - " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n")

run_multiclass_experiment(path, 0, 1)

for window in range(15, 46, 15):
    for timestep in range(2, 9):
      run_multiclass_experiment(path, window, timestep)

4 LSTM-слоя

In [ ]:
!mkdir -p /content/drive/MyDrive/Thesis/Results
all_steps = None
all_results = None
log_file = "/content/drive/MyDrive/Thesis/Results/multiclass_log_v1_1h.txt"

with open(log_file, "a") as f:
    f.write("Training Log - " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n")

run_multiclass_experiment(path, 0, 1, "LSTM4")

for window in range(15, 46, 15):
    for timestep in range(2, 9):
      run_multiclass_experiment(path, window, timestep, "LSTM4")